# Database Joins 1: Combining JIRA With GitHub

For this notebook, our research question is as follows:

<b style="color:green">Are there Liferay Portal Patches (LPP) tickets that are currently stuck in review?</b>

In order to answer this question, this notebook introduces database joins. However, rather than teaching you what a database join looks like in SQL, it instead shows you a very simple database join in code while presenting some of the underlying implementation concepts.

At the end of this notebook, we will have a script that combines data retrieved from JIRA with data retrieved from GitHub, and the reader will have an improved understanding of what goes into a join.

## Prerequisites

The following cell attempts to use `conda` and `pip` to install the libraries that are used by this notebook. If the output indicates that additional items were installed, you will need to restart the kernel after the installation completes before you can run the later cells in the notebook.

In [ ]:
!conda install -y numpy pandas pytz requests ujson
!pip install dateparser

## Notebook Imports

In [ ]:
from __future__ import print_function

from collections import defaultdict, namedtuple
import dateparser
from datetime import date, datetime
import hashlib
import numpy as np
import os
import pandas as pd
import pytz
import re
import requests
import six
import sys
import subprocess
import ujson as json

## Save Raw Data

Before we start, we'll make sure that we establish one rule for this script and all future scripts: we will save all raw data with timestamps.

In [ ]:
today = date.today()
now = datetime.now(pytz.utc)

This is important, because one of the more common things to do as a developer is to retrieve the data, extract only the information you need, and then discard the data you do not need. However, unless you have some terms of service agreement restricting what data you are allowed to retain, do not discard the raw data! Raw data is a starting point that speeds up the creation of many different application prototypes.

We could serialize all the data as a single JSON object. In this case, however, we're going to adopt a slightly less conventional data format, as it will make the future tutorials talking about database indices much easier to explain. The file format is as follows for each key-value pair:

```
primkey [TAB] index_field_1 [TAB] index_field_2 [TAB] ... [TAB] index_field_n [TAB] row_value
```

In [ ]:
def get_file_name(cache_name, suffix):
    base_name = os.path.basename(cache_name)
    subfolder_name = os.path.dirname(cache_name)
    folder_name = 'rawdata/%s' % subfolder_name

    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    return '%s/%s_%s.%s' % (folder_name, today.isoformat(), base_name, suffix)

In [ ]:
def save_row(outfile, keys, row_value):
    for key in keys:
        outfile.write(json.dumps(key))
        outfile.write('\t')

    outfile.write(json.dumps(row_value))
    outfile.write('\n')
    
def save_raw_dict(cache_name, raw_dict, index_fields=[]):
    file_name = get_file_name(cache_name, '.json')

    with open(file_name, 'w') as outfile:
        for primkey, row_value in raw_dict.items():
            keys = [primkey] + [row[field] if field in row else None for field in index_fields]
            save_row(outfile, keys, row_value)
    
    return load_raw_dict(cache_name)

In [ ]:
def load_row(line):
    row = line.split('\t')
    keys = [json.loads(key) for key in row[0:-1]]
    return keys, json.loads(row[-1])

def load_raw_dict(cache_name):
    file_name = get_file_name(cache_name, '.json')

    if not os.path.isfile(file_name):
        return None

    raw_dict = {}
    indexed_data = defaultdict(list)
    
    with open(file_name) as infile:
        for line in infile:
            keys, row_value = load_row(line)

            primkey = keys[0]
            raw_dict[primkey] = row_value

            index_fields = tuple(keys[1:])
            
            if len(index_fields) > 0:
                indexed_data[index_fields].append(row_value)

    if len(indexed_data) == 0:
        return raw_dict
            
    return raw_dict, indexed_data

## Load Data from JIRA, Part 1

Let's keep our question in mind during each step.

<b style="color:green">Are there Liferay Portal Patches (LPP) tickets that are currently stuck in review?</b>

Before we can answer the question of whether anything is *stuck in review*, we will need to answer the broader question of what is *in review*. We'll start by looking at what's in review in JIRA.

### Login to JIRA

Because [issues.liferay.com](https://issues.liferay.com/) does not have OAuth support, we will need to find a different way to connect to our JIRA installation. The simplest way is to simply login to JIRA.

There are a lot of secure ways to specify your username and password, but for the sake of this script, we'll use the most insecure way possible: a plain text file. Namely, the `.gitconfig` in your user's home folder. If you want to use a different strategy that's less global (a plain text JSON file in the same folder as this script, for example) or more secure, just change the implementation of the two functions below.

In [ ]:
def get_config(key):
    try:
        return subprocess.check_output(['git', 'config', key]).strip().decode('utf8')
    except:
        return None

def set_config(key, value):
    subprocess.call(['git', 'config', '--global', key, value])
    subprocess.call(['git', 'config', '--global', key, value])

Assuming you are using the default implementation, set your username and password inside of the `.gitconfig` located in your user's home folder by running the following two commands in a command line window.

```
git config --global jira.session-username JIRA_USERNAME
git config --global jira.session-password JIRA_PASSWORD
```

The following cell will read it in and confirm that it exists.

In [ ]:
jira_username = get_config('jira.session-username')
jira_password = get_config('jira.session-password')

assert(jira_username is not None)
assert(jira_password is not None)

The following cell will use this information and attempt to login to JIRA and confirm that it has a valid session, which will confirm that the credentials you saved are valid. It will also save this session information so that it can reuse it later without constantly relogging in.

In [ ]:
jira_base_url = 'https://issues.liferay.com/rest'

def get_jira_cookie():
    jira_cookie = None

    jira_cookie_name = None
    jira_cookie_value = None

    try:
        jira_cookie_name = get_config('jira.session-cookie-name')
        jira_cookie_value = get_config('jira.session-cookie-value')
    except:
        pass
    
    if jira_cookie_name is not None and jira_cookie_value is not None:
        jira_cookie = {
            jira_cookie_name: jira_cookie_value
        }

        r = requests.get(jira_base_url + '/auth/1/session', cookies=jira_cookie)

        if r.status_code != 200:
            jira_cookie = None

    if jira_cookie is not None:
        return jira_cookie
        
    post_json = {
        'username': jira_username,
        'password': jira_password
    }

    r = requests.post(jira_base_url + '/auth/1/session', json=post_json)

    if r.status_code != 200:
        print('Invalid login')

        return None

    response_json = r.json()

    jira_cookie_name = response_json['session']['name']
    jira_cookie_value = response_json['session']['value']

    set_config('jira.session-cookie-name', jira_cookie_name)
    set_config('jira.session-cookie-value', jira_cookie_value)

    jira_cookie = {
        jira_cookie_name: jira_cookie_value
    }

    return jira_cookie

In [ ]:
assert(get_jira_cookie() is not None)

### Retrieve JIRA Issues

Now that we have a valid login, our next step is to use the JIRA API to retrieve tickets. If you've interacted with JIRA before, you know that it has its own query language (JQL). It turns out there is a simple search API that allows you to submit the JQL and all the matching issues are returned as JSON. Since the API is fairly simple, we implement it here.

In [ ]:
def retrieve_jira_issues(jql):
    jira_cookie = get_jira_cookie()

    if jira_cookie is None:
        return []

    start_at = 0

    post_json = {
        'jql': jql,
        'startAt': start_at
    }

    r = requests.post(jira_base_url + '/api/2/search', cookies=jira_cookie, json=post_json)

    if r.status_code != 200:
        return {}

    response_json = r.json()

    issues = {}
    
    for issue in response_json['issues']:
        issues[issue['key']] = issue

    while start_at + response_json['maxResults'] < response_json['total']:
        start_at += response_json['maxResults']
        post_json['startAt'] = start_at

        r = requests.post(jira_base_url + '/api/2/search', cookies=jira_cookie, json=post_json)

        if r.status_code != 200:
            return issues

        response_json = r.json()

        for issue in response_json['issues']:
            issues[issue['key']] = issue

    return issues

Now that we have something that can retrieve JIRA issues, all we need is to actually create our JQL and then run the search. This is the JQL we'll use for regular Liferay Portal Patch (LPP) issues that are in review.

In [ ]:
in_review_jql = '''
    project = LPP AND
    type not in ("SME Request", "SME Request SubTask") AND
    status = "In Review"
    order by key
'''

Let's go ahead and retrieve those results.

In [ ]:
jql_hashes = load_raw_dict('jql_hashes')

if jql_hashes is None:
    jql_hashes = {}

def get_jql_hashed_name(base_name, jql):
    jql_hash = None
    
    for key, value in jql_hashes.items():
        if value == jql:
            jql_hash = key
            break

    if jql_hash is None:            
        digester = hashlib.md5()
        digester.update(jql)
        jql_hash = digester.hexdigest()

        jql_hashes[jql_hash] = jql

        save_raw_dict('jql_hashes', jql_hashes)
    
    return '%s/%s' % (jql_hash, base_name)

def get_jira_issues(jql):
    base_name = get_jql_hashed_name('jira_issues', jql)

    jira_issues = load_raw_dict(base_name)

    if jira_issues is not None:
        print('Loaded cached JIRA search')
        return jira_issues

    print('Executing JIRA search')

    jira_issues = retrieve_jira_issues(in_review_jql)
    jira_issues = save_raw_dict(base_name, jira_issues)
    return jira_issues

In [ ]:
if __name__ == '__main__':
    jira_issues = get_jira_issues(in_review_jql)
else:
    jira_issues = {}

Looking at JSON is a bit tedious, so we'll take a look at a subset of fields in a way that resembles the view you get when you run JQL via the web browser.

In [ ]:
JIRAIssue = namedtuple(
    'JIRAIssue',
    ['ticket_key', 'region', 'status', 'assignee', 'summary']
)

def get_jira_tuple(issue):
    region_field_name = 'customfield_11523'

    regions = ['']

    if region_field_name in issue['fields']:
        regions = [region['value'] for region in issue['fields'][region_field_name]]

    return JIRAIssue(
        ticket_key=issue['key'],
        region=regions[0],
        status=issue['fields']['status']['name'],
        assignee=issue['fields']['assignee']['displayName'],
        summary=issue['fields']['summary']
    )

In [ ]:
pd.DataFrame([get_jira_tuple(issue) for issue in jira_issues.values()])

## Load Data from GitHub, Part 1

Let's remind ourselves of our original question.

<b style="color:green">Are there Liferay Portal Patches (LPP) tickets that are currently stuck in review?</b>

What is this concept of "in review"? Well, being "in review" actually means that your code has been written and you are now waiting on another team member to look at your changes, providing a sanity check from someone who is looking at the solution with fresh eyes.

Where do these sanity check reviews occur? They occur on GitHub. This means that we will want to bring in the GitHub data set in order to answer our question.

### Login to GitHub

Luckily, [api.github.com](https://developer.github.com/v3/) does have OAuth support, so we'll want to request an OAuth token from GitHub and leverage it in our script.

* [Creating a personal access token for command line](https://help.github.com/articles/creating-a-personal-access-token-for-the-command-line/)

Assuming you are using the default implementation for configuration values provided by this notebook (mentioned earlier when setting up JIRA access), set your OAuth token inside of the `.gitconfig` located in your user's home folder by running the following command in a command line window.

```
git config --global github.oauth-token GITHUB_OAUTH_TOKEN
```

If you customized it, do whatever you need to get the configuration value persisted. The following cell will read it in and confirm that it exists.

In [ ]:
github_oauth_token = get_config('github.oauth-token')

assert(github_oauth_token is not None)

However, it's not enough that it exists. We should confirm that the token works on the repositories that are related to Liferay. We'll use the `liferay/liferay-portal` and `liferay/liferay-portal-ee` repositories as a way to validate.

In [ ]:
github_base_url = 'https://api.github.com'

def is_repository_accessible(reviewer_url):
    print('Validating OAuth token against %s' % reviewer_url)

    headers = {
        'user-agent': 'python checklpp.py',
        'authorization': 'token %s' % github_oauth_token
    }

    api_path = '/repos/%s' % reviewer_url
    
    r = requests.get(github_base_url + api_path, headers=headers)
    
    return r.status_code == 200

assert(is_repository_accessible('liferay/liferay-portal'))
assert(is_repository_accessible('liferay/liferay-portal-ee'))

## Detour: Join Strategies

We currently have one data set (a list of JIRA tickets that are in review), and our next step is to think about how we can answer our actual question.

<b style="color:green">Are there Liferay Portal Patches (LPP) tickets that are currently stuck in review?</b>

Now that we've identified which LPP tickets are currently in review, our next step is to identify pull requests that are stuck in review. So, that leaves us with the following question: how should we implement this?

### Goals of Query Optimization

What will happen from here is you ask the database to provide an explanation of what it's doing for your query, you work some magic to make this explanation look better (add database indices, modify the query), and you commit those changes to the codebase.

* [Query plan](https://en.wikipedia.org/wiki/Query_plan)

Our definition of "better" comes from interpreting certain aspects of the query plan as necessarily worse than what we expected. Our expectations often come from definitions that are provided by various database vendors on the ideal query plan.

* [MySQL explain plan](https://dev.mysql.com/doc/refman/5.6/en/explain-output.html)
* [Oracle explain plan](https://docs.oracle.com/database/121/TGSQL/tgsql_interp.htm)
* [PostgreSQL explain plan](http://www.postgresql.org/docs/9.4/static/using-explain.html)
* [SQL Server explain plan](https://technet.microsoft.com/en-us/library/ms178071%28v=sql.105%29.aspx)

When you run into a slow query and you ask for it to provide an explanation of its approach, the output you receive is the query plan derived by the query optimizer.

* [Query optimization](https://en.wikipedia.org/wiki/Query_optimization)

Query optimization plans essentially evaluate various ways of correctly answering the query described by the SQL statement while also minimizing the cost of loading the query. For a database, this notion of "cost" can be summarized as use of memory (in particular, pages getting swapped in and out of active memory), disk I/O, and network I/O.

* [Relative time cost of computer operations](http://norvig.com/21-days.html#answers)

The result of this optimized plan particularly easy to understand in Microsoft SQL Server, where it gives you a graphical representation of its query plan, and it estimates what percentage of the time spent processing the query will be spent on that specific aspect of it.

* [SQL Server query execution plans](http://www.sqlshack.com/sql-server-query-execution-plans-understanding-reading-plans/)
* [SQL Server graphical plan icons](https://technet.microsoft.com/en-us/library/ms175913%28v=sql.105%29.aspx)

### Goals of Join Strategies

Taking a step back, what are we doing is that we need to derive new information from two separate data sources. If these two data sources were both tables, then if these tables have columns that refer to the same abstract concept (or perhaps a third "mapping table" that describes that abstract concept), then we would join these two 

* [Khan Academy: Joining Related Tables](https://www.khanacademy.org/computing/computer-programming/sql/relational-queries-in-sql/p/joining-related-tables)

We've identified all of the JIRA issues that are in review. Presumably, we want to then combine this with GitHub data. What algorithm will we implement to achieve this? It turns out there are three basic strategies for computing a join: a nested loop join, a hash join, and a sort-merge join.

* [Join methods and subqueries](http://www.orafaq.com/tuningguide/join%20methods.html)

Sort-merge deserves its own discussion (which we'll do in the next tutorial), and it will make more sense after we're finished than before we've finished. So we'll look first at how a nested loop join and a hash join relate to our problem.

### Nested Loop Join

A simple solution to this problem would be to iterate over each of our pull requests and then check each JIRA ticket to see if the JIRA ticket references the pull request. So for every GitHub pull request, you would check every JIRA ticket. This strategy is known as a **nested loop join**.

A nested loop join is a join where the query optimizer decides the best way to accomplish the join is to designate one table as the "outer table" (no connection to the notion of an outer join) and the other table as the "inner table", structured in much the same way as a for loop.

* [Nested loop join](https://en.wikipedia.org/wiki/Nested_loop_join)

How fast is a nested loop join? Let $m$ be the size of the JIRA table and $n$ be the size of the GitHub table. Regardless of which table you choose for the outer table, a nested loop join would have an expected runtime and a worst-case runtime of $O(m \cdot n)$.

### Hash Join

Another solution to this problem would be to first load all the pull requests into a lookup data structure. From there, iterate over the JIRA tickets and match the pull requests to the hash table. So for every JIRA ticket, you would perform a number of lookups against our data structure not based on either tables size (so, effectively a constant). This strategy is known as a **hash join**.

A hash join is a join where the query optimizer decides the best way to accomplish the join is to build a lookup table on the smaller table (because it's more likely that a smaller table can fit into memory), with a popular choice being a hash table, and then iterate over the larger table.

* [Hash join](https://en.wikipedia.org/wiki/Hash_join)

How fast is a hash join? Let $m$ be the size of the JIRA table and $n$ be the size of the GitHub table. Note that hash tables have an $O(1)$ expected access and insertion time, but an $O(n)$ worst case access and insertion time (due to hash collisions, table resizes). So in practice, we have an expected runtime that is $O(m) + O(n)$ and a worst-case runtime that is $O(m \cdot n) + O(n^2)$.

## Load Data from GitHub, Part 2

Now that we have a valid login, we can proceed with our question.

<b style="color:green">Are there Liferay Portal Patches (LPP) tickets that are currently stuck in review?</b>

As noted previously, before we can answer the question of whether anything is *stuck in review*, we will need to answer the broader question of what is *in review*. Now that we know what's in review in JIRA, we also want to what's in review on GitHub. Then, we will want to join these two tables together.

Between a nested loop join and a hash join, the hash join has the better expected runtime. So how can we perform a hash join? We need to create a lookup data structure containing all the pull requests using some key.

### Retrieve GitHub Pull Requests

Unfortunately, given the number of unique users repositories that are involved in reviewing Liferay pull requests, it's not practical to make an API call against every user and every repository. Instead, we'll want to do this on-demand based on which pull requests we know exist.

If you visit `/repos/USERNAME/REPOSITORY/pulls` without a pull ID, GitHub will return all currently open pull requests. The approach below uses this API in an attempt to reduce the number of API requests to GitHub, because this will fetch all pull requests that are open in a single request.

Following our principle of keeping any data we retrieve, we save all of these pull requests, and then we request any additional pull requests that are closed and save those as well. Because our end goal is a hash join, we'll load this table as a map or dictionary where the key is the GitHub URL, because that's what will be present in JIRA fields.

In [ ]:
def retrieve_pull_requests(reviewer_url, pull_request_ids=[]):
    print('Checking pull requests waiting on %s' % reviewer_url)

    headers = {
        'user-agent': 'python checklpp.py',
        'authorization': 'token %s' % github_oauth_token
    }

    api_path = '/repos/%s/pulls' % reviewer_url

    r = requests.get(github_base_url + api_path, headers=headers)

    if r.status_code != 200:
        return {}

    new_pull_requests_list = r.json()
    new_pull_requests = {
        pull_request['html_url']: pull_request
            for pull_request in new_pull_requests_list
    }

    for pull_request_id in pull_request_ids:
        github_url = 'https://github.com/%s/pull/%s' % (reviewer_url, pull_request_id)

        if github_url in new_pull_requests:
            continue

        api_path = '/repos/%s/pulls/%s' % (reviewer_url, pull_request_id)

        r = requests.get(github_base_url + api_path, headers=headers)

        if r.status_code != 200:
            continue

        new_pull_requests[github_url] = r.json()

    return new_pull_requests

For now, we'll take a look at all the pull requests open against `liferay/liferay-portal-ee`.

In [ ]:
def get_open_backports():
    open_backports = load_raw_dict('open_backports')

    if open_backports is not None:
        print('Loaded cached open backports')
        return open_backports
        
    open_backports = retrieve_pull_requests('liferay/liferay-portal-ee')
    open_backports = save_raw_dict('open_backport_pulls', open_backports)
    return open_backports

In [ ]:
if __name__ == '__main__':
    open_backports = get_open_backports()
else:
    open_backports = {}

In [ ]:
GHPullRequest = namedtuple(
    'GHPullRequest',
    ['submitter', 'pull_id', 'branch', 'created_at', 'updated_at', 'closed_at', 'state', 'github_url']
)

def get_github_tuple(pull_request):
    pull_id = '%s/%s#%d' % (
        pull_request['base']['user']['login'],
        pull_request['base']['repo']['name'],
        pull_request['number']
    )
    
    return GHPullRequest(
        submitter=pull_request['user']['login'],
        pull_id=pull_id,
        branch=pull_request['base']['ref'],
        created_at=pull_request['created_at'],
        updated_at=pull_request['updated_at'],
        closed_at=pull_request['closed_at'],
        state=pull_request['state'],
        github_url=pull_request['html_url']
    )

In [ ]:
pd.DataFrame([get_github_tuple(pull_request) for pull_request in open_backports.values()])

## Detour: Mapping Tables

Now that we have a function that can retrieve GitHub pull request metadata, you could say that in addition to having a JIRA table and a GitHub table. However, how can we join the two together?

The answer is a strategy that you see implemented in Liferay Service Builder. This strategy is known as a mapping table, named for how it explicitly creates many-to-many mappings.

* [Understanding Mapping Tables](https://stackoverflow.com/questions/6453462/mysql-understanding-mapping-tables)

### Mapping Table Extensions

Let's assume you have two tables, JIRA and GitHub You can use a mapping table in order to document the primary keys of JIRA and how they relate to the primary keys on GitHub.

What kinds of question can you answer with a mapping table? Simplistically, you could fetch all JIRA tickets corresponding to a specific GitHub pull request and all GitHub pull requests corresponding to a specific JIRA ticket. If you perform a three-way join, you could get all JIRA tickets and all GitHub pull requests that reference each other.

To answer these questions, you have to think more carefully about how to optimize these queries. A simple solution that works on smaller data sets is to load the entire mapping table into memory and perform hash joins on both sides.

However, database vendors have found that you can technically do better. There is a logical extension of a mapping table that brings together more than two keys, and this extension is referred to as a star schema. These are actually common in data warehouses, and databases can be configured (usually through the use of specialized database index types) to drastically improve query cost plans against star schemas.

* [Star schema](https://en.wikipedia.org/wiki/Star_schema)

## Load Data from GitHub, Part 3

So now that we have our lookup data structure, what do we need to answer our question?

<b style="color:green">Are there Liferay Portal Patches (LPP) tickets that are currently stuck in review?</b>

Well, while we have a lookup data structure, it turns out we don't actually have a GitHub table yet, only a subset of the GitHub table corresponding to pull requests sent against `liferay/liferay-portal-ee`! So our next step is to generate the relevant portion of our GitHub table.

### Identify GitHub Pulls Related to JIRA Issues

So while we have a utility method that allows us to retrieve GitHub pull request histories, we don't actually have most of that information. So how do we identify which ones to retrieve?

Our first step is to iterate over each JIRA ticket and extract the pull requests contained in the ticket. Once we have that information, we'll be able to use our lookup data structure.

In [ ]:
def extract_jira_pull_request_urls(jira_issues):
    issues_by_request = defaultdict(set)
    requests_by_issue = defaultdict(set)
    requests_by_reviewer = defaultdict(set)

    for jira_key, jira_issue in jira_issues.items():
        for value in jira_issue['fields'].values():
            if not isinstance(value, six.string_types):
                continue

            for github_url in re.findall('https://github.com/[^\s]*/pull/[\d]+', value):
                requests_by_issue[jira_key].add(github_url)
                issues_by_request[github_url].add(jira_key)

    return issues_by_request, requests_by_issue

Now, let's pass the data the results of our previous JIRA search to this function.

In [ ]:
def get_jira_pull_request_urls(jql):
    base_name_1 = get_jql_hashed_name('issues_by_request', jql)
    base_name_2 = get_jql_hashed_name('requests_by_issue', jql)
    
    issues_by_request = load_raw_dict(base_name_1)
    requests_by_issue = load_raw_dict(base_name_2)

    if issues_by_request is not None and requests_by_issue is not None:
        print('Loaded cached JIRA to GitHub mapping')
        return issues_by_request, requests_by_issue
    
    jira_issues = get_jira_issues(jql)
    issues_by_request, requests_by_issue = extract_jira_pull_request_urls(jira_issues)

    issues_by_request = save_raw_dict(base_name_1, issues_by_request)
    requests_by_issue = save_raw_dict(base_name_2, requests_by_issue)
    
    return issues_by_request, requests_by_issue

In [ ]:
if __name__ == '__main__':
    issues_by_request, requests_by_issue = get_jira_pull_request_urls(in_review_jql)
else:
    issues_by_request = {}
    requests_by_issue = {}

In [ ]:
JIRAGitHubMapping = namedtuple('JIRAGitHubMapping', ['jira_key', 'github_url'])

In [ ]:
pd.DataFrame([
    JIRAGitHubMapping(jira_key=jira_key, github_url=github_url)
        for jira_key, github_urls in requests_by_issue.items()
            for github_url in github_urls
])

As you can see, it turns out that what we've done is equivalent to building a mapping table between the JIRA tickets (represented with their issue key) and the GitHub pull requests (represented with their URL).

### Retrieve GitHub Pulls Related to JIRA Issues

Now, we'll want to fetch all the metadata associated with all those pull requests. We'll also want a quick way to separate the open/active pull requests from the closed/inactive pull requests so that we don't have to re-derive that metadata later on when we attempt to identify stuck pull requests.

In [ ]:
def retrieve_related_pull_requests(issues_by_request):
    requests_by_reviewer = defaultdict(set)
    
    for github_url in issues_by_request.keys():
        reviewer_url = github_url[19:github_url.rfind('/pull/')]
        requests_by_reviewer[reviewer_url].add(github_url[github_url.rfind('/')+1:])

    related_pull_requests = {}

    for reviewer_url, pull_request_ids in sorted(requests_by_reviewer.items()):
        new_pull_requests = retrieve_pull_requests(reviewer_url, pull_request_ids)
        related_pull_requests.update(new_pull_requests)

    return related_pull_requests

Let's go ahead and use our mapping tables to populate the subset of the GitHub table corresponding to only (1) currently active pull requests, or (2) closed pull requests corresponding to a Liferay Portal Patch ticket that is currently in review.

In [ ]:
def get_related_pull_requests(jql):
    base_name = get_jql_hashed_name('related_pull_requests', jql)
    
    related_pull_requests = load_raw_dict(base_name)

    if related_pull_requests is not None:
        print('Loaded cached pull request metadata')
        return related_pull_requests

    issues_by_request, requests_by_issue = get_jira_pull_request_urls(jql)

    related_pull_requests = retrieve_related_pull_requests(issues_by_request)
    related_pull_requests = save_raw_dict(base_name, related_pull_requests)
    
    return related_pull_requests

In [ ]:
if __name__ == '__main__':
    related_pull_requests = get_related_pull_requests(in_review_jql)
else:
    related_pull_requests = {}

In [ ]:
pd.DataFrame([get_github_tuple(pull_request) for pull_request in related_pull_requests.values()])

## Detour: Derived Tables

As seen in the result of executing the previous code cells, a lot of what we're doing can be conceptualized as building tables that satisfy our search parameters. In the case of JIRA, it's all issues matching our query string. In the case of GitHub, it's all open pull requests alongside all of the pull requests that are tied to an LPP ticket.

Just as lists of results can be conceptualized as a table, so too are things that work with regular SQL. More explicitly, whenever you're working with a database and you execute SQL, the result can be understood to be another table, whether this table consists of a single value, a single row, or multiple rows.

* [Relational algebra](https://en.wikipedia.org/wiki/Relational_algebra)

At a conceptual level, running database queries is taking existing tables and generating (or deriving) new tables that better fit the question you are trying to answer.

### Persisting Derived Tables

The last point is worth emphasizing, as it comes up in upgrade performance tuning. In short, running database queries always generates new tables. More explicitly, the database will have gone through all of the effort required to create a table when executing your query, skipping only the persistence step.

* [The difference between subqueries and derived tables in SQL](https://www.xaprb.com/blog/2005/09/26/sql-subqueries-and-derived-tables/)

Looking at this a little differently, imagine that we decided to remove the code that invoked the `load_raw_dict` and `save_raw_dict` functions. Would this have noticeably improved the performance of our code? Not really, because the retrieval from JIRA is slower than writing the data to disk. That's equivalent to the cost difference between persisting the data and not persisting it.

Having the derived table be saved as an permanent table also allows you to add metadata, such as indices, that will improve the performance on repeated queries. So, in cases where you are operating on different subsets of a larger subset of the data, and this larger subset is expensive to compute, it's actually very sensible to save your derived table to reduce query execution time.

## Perform JIRA-GitHub Join

We now have raw data from both JIRA and GitHub. How can we proceed in answering our original question?

<b style="color:green">Are there Liferay Portal Patches (LPP) tickets that are currently stuck in review?</b>

One way that we can define "stuck in review" is an LPP ticket that has a GitHub pull request that has remained open for a long time. Another way that we can define "stuck in review" is an LPP ticket that has no open GitHub pull requests, but is still in review even after the last pull request closed.

Both of these require us to perform a join between our two tables.

### Open LPP Pull Request Count

Both of these questions require us to join the two tables together, and then filter the end result. What we treat as idle time is different depending on whether there are zero open pull requests or at least one open pull request.

Our job would be easier if we knew how many pull requests that were still open for each JIRA ticket, which we could express with the following SQL.

```
SELECT Jira.ticket_key AS jira_key,
       count(*)        AS open_count
FROM   Jira
       LEFT OUTER JOIN Jira_GitHub
                    ON Jira.ticket_key = Jira_GitHub.jira_key
       INNER JOIN GitHub
               ON Jira_GitHub.github_url = GitHub.github_url
WHERE  GitHub.state = 'open'
GROUP  BY Jira.ticket_key
```

So we're going to persist the result of this computation to make future computation easier.

In [ ]:
def get_jira_github_join(jql):
    base_name = get_jql_hashed_name('jira_github_join', jql)
    
    jira_github_join = load_raw_dict(base_name)

    if jira_github_join is not None:
        print('Loaded cached JIRA-GitHub join result')
        return jira_github_join

    jira_issues = get_jira_issues(jql)
    issues_by_request, requests_by_issue = get_jira_pull_request_urls(jql)
    related_pull_requests = get_related_pull_requests(jql)

    jira_github_join = {
        jira_key: {
            'jira': jira_issues[jira_key],
            'github': [related_pull_requests[github_url] for github_url in github_urls]
        }
        for jira_key, github_urls in requests_by_issue.items()
            if github_url in related_pull_requests
    }
    
    jira_github_join = save_raw_dict(base_name, jira_github_join)
    
    return jira_github_join

def get_github_open_count(jql):
    base_name = get_jql_hashed_name('github_open_count', jql)
    
    github_open_count = load_raw_dict(base_name)

    if github_open_count is not None:
        print('Loaded cached JIRA-GitHub join count result')
        return github_open_count

    jira_github_join = get_jira_github_join(jql)

    github_open_count = {
        jira_key: len([pull_request for pull_request in join_result['github'] if pull_request['state'] == 'open'])
            for jira_key, join_result in jira_github_join.items()
    }
    
    github_open_count = save_raw_dict(base_name, github_open_count)
    
    return github_open_count

In [ ]:
if __name__ == '__main__':
    github_open_count = get_github_open_count(in_review_jql)
else:
    github_open_count = {}

In [ ]:
GitHubOpenCount = namedtuple('GitHubOpenCount', ['jira_key', 'open_count'])

In [ ]:
pd.DataFrame([
    GitHubOpenCount(jira_key=jira_key, open_count=open_count)
        for jira_key, open_count in github_open_count.items()
])

### Stuck in Review: At Least One Open Pull

Now that we have this table, we can look for the creation time on any open pull requests for any ticket with at least one open pull request.

```
SELECT GitHubOpenCount.jira_key,
       GitHub.*
FROM   GitHubOpenCount
       INNER JOIN Jira_GitHub
               ON Jira.ticket_key = Jira_GitHub.jira_key
       INNER JOIN GitHub
               ON Jira_GitHub.github_url = GitHub.github_url
WHERE  GitHubOpenCount.open_count > 0
       AND GitHub.state = 'open'
GROUP  BY GitHubOpenCount.jira_key
```

In [ ]:
def get_github_idle_tickets(jql):
    base_name = get_jql_hashed_name('github_idle_tickets', jql)
    
    github_idle_tickets = load_raw_dict(base_name)

    if github_idle_tickets is not None:
        print('Loaded cached list of tickets idle on GitHub')
        return github_idle_tickets

    github_open_count = get_github_open_count(jql)    
    jira_github_join = get_jira_github_join(jql)
    
    github_idle_tickets = {
        jira_key: {
            'jira': join_result['jira'],
            'github': [pull_request for pull_request in join_result['github'] if pull_request['state'] == 'open']
        }
        for jira_key, join_result in jira_github_join.items() if github_open_count[jira_key] > 0
    }

    github_idle_tickets = save_raw_dict(base_name, github_idle_tickets)
    
    return github_idle_tickets

In [ ]:
if __name__ == '__main__':
    github_idle_tickets = get_github_idle_tickets(in_review_jql)
else:
    github_idle_tickets = {}

In [ ]:
JiraGitHubLookup = namedtuple(
    'JiraGitHubLookup',
    list(JIRAIssue._fields) + list(GHPullRequest._fields)
)

def get_jira_github_tuple(jira_key, jira_issue, pull_request):
    jira_tuple = get_jira_tuple(jira_issue)
    github_tuple = get_github_tuple(pull_request)

    combined_columns = jira_tuple._asdict()
    combined_columns.update(github_tuple._asdict())
    
    new_tuple = JiraGitHubLookup(**combined_columns)

    return new_tuple

In [ ]:
pd.DataFrame([
    get_jira_github_tuple(jira_key, join_result['jira'], pull_request)
        for jira_key, join_result in github_idle_tickets.items()
            for pull_request in join_result['github']
])

### Stuck In Review: No Open Pulls

We could also look for the updated time on the last closed pull request for any ticket with zero open pull requests.

```
SELECT GitHubOpenCount.jira_key as jira_key,
       max(GitHub.closed_at) AS closed_at
FROM   GitHubOpenCount
       INNER JOIN Jira_GitHub
               ON Jira.ticket_key = Jira_GitHub.jira_key
       INNER JOIN GitHub
               ON Jira_GitHub.github_url = GitHub.github_url
WHERE  GitHubOpenCount.open_count = 0
       AND GitHub.state = 'closed'
GROUP  BY GitHubOpenCount.jira_key
```

In [ ]:
def get_jira_idle_tickets(jql):
    base_name = get_jql_hashed_name('jira_idle_tickets', jql)
    
    jira_idle_tickets = load_raw_dict(base_name)

    if jira_idle_tickets is not None:
        print('Loaded cached list of tickets idle on JIRA')
        return jira_idle_tickets

    github_open_count = get_github_open_count(jql)    
    jira_github_join = get_jira_github_join(jql)
    
    github_closed_pulls = {
        key: {
            'jira': join_result['jira'],
            'github': [
                pull_request
                    for pull_request in join_result['github'] if pull_request['state'] == 'closed'
            ]
        }
        for key, join_result in jira_github_join.items() if github_open_count[key] == 0
    }
    
    jira_idle_tickets = {
        key: {
            'jira': join_result['jira'],
            'github': max(join_result['github'], key=lambda x: x['closed_at'])
        }
        for key, join_result in github_closed_pulls.items()
    }

    jira_idle_tickets = save_raw_dict(base_name, jira_idle_tickets)
    
    return jira_idle_tickets

In [ ]:
if __name__ == '__main__':
    jira_idle_tickets = get_jira_idle_tickets(in_review_jql)
else:
    jira_idle_tickets = {}

In [ ]:
pd.DataFrame([
    get_jira_github_tuple(jira_key, join_result['jira'], join_result['github'])
        for jira_key, join_result in jira_idle_tickets.items()
])

## Add Computed Fields

We now have a list of LPP tickets that have at least one open GitHub pull request. We also have a list of LPP tickets that have no open GitHub pull requests but are still in review, and the last closed pull request associated with that LPP ticket. Let's come back to our question.

<b style="color:green">Are there Liferay Portal Patches (LPP) tickets that are currently stuck in review?</b>

While we could look at the dates to determine how long everything has been open, it'd be nice if that value was displayed for us in a different way. That would make it much easier to see, at a glance, what's still idle.

### Add Idle Time

All we're doing now is adding a computed value to our table.

In [ ]:
def get_time_delta_as_days(time_delta):
    return float(time_delta.days) + float(time_delta.seconds) / (60 * 60 * 24)

new_fields = list(JiraGitHubLookup._fields) + ['open_time_days', 'idle_time_days']
removed_fields = ['created_at', 'updated_at', 'closed_at']

for removed_field in removed_fields:
    new_fields.remove(removed_field)

JiraGitHubLookupIdleTime = namedtuple('JiraGitHubLookupIdleTime', new_fields)

def get_jira_github_idle_time_tuple(jira_key, jira_issue, pull_request):
    old_tuple = get_jira_github_tuple(jira_key, jira_issue, pull_request)
    old_values = old_tuple._asdict()

    for removed_field in removed_fields:
        del old_values[removed_field]
    
    created_at = dateparser.parse(pull_request['created_at'])
    updated_at = dateparser.parse(pull_request['updated_at'])

    closed_at = pull_request['closed_at']
    
    if closed_at is None:
        open_time_days = get_time_delta_as_days(now - created_at)
        idle_time_days = get_time_delta_as_days(now - updated_at)
    else:
        closed_at = dateparser.parse(pull_request['closed_at'])

        open_time_days = None
        idle_time_days = get_time_delta_as_days(now - closed_at)
    
    new_tuple = JiraGitHubLookupIdleTime(
        open_time_days=open_time_days,
        idle_time_days=idle_time_days,
        **old_values
    )

    return new_tuple

### Stuck in Review: At Least One Open Pull

In [ ]:
pd.DataFrame([
    get_jira_github_idle_time_tuple(jira_key, join_result['jira'], pull_request)
        for jira_key, join_result in github_idle_tickets.items()
            for pull_request in join_result['github']
])

### Stuck in Review: No Open Pulls

In [ ]:
pd.DataFrame([
    get_jira_github_idle_time_tuple(jira_key, join_result['jira'], join_result['github'])
        for jira_key, join_result in jira_idle_tickets.items()
])

## Export the Results for Grow

Now that we have all the data, it's time to export it so that something outside of this notebook can use it. For simplicity, we'll export the data as Javascript variables, which can then be accessed by other Javascript on the page.

In [ ]:
if __name__ == '__main__':
    github_idle_tickets_list = [
        get_jira_github_idle_time_tuple(jira_key, join_result['jira'], pull_request)._asdict()
            for jira_key, join_result in github_idle_tickets.items()
                for pull_request in join_result['github']
    ]
    
    jira_idle_tickets_list = [
        get_jira_github_idle_time_tuple(jira_key, join_result['jira'], join_result['github'])._asdict()
            for jira_key, join_result in jira_idle_tickets.items()
    ]
    
    with open('%s_idle_ticket_data.json' % today.isoformat(), 'w') as outfile:
        idle_ticket_data = {
            'lastUpdated': now.isoformat(),
            'githubIdleTicketsList': github_idle_tickets_list,
            'jiraIdleTicketsList': jira_idle_tickets_list
        }
        
        json.dump(idle_ticket_data, outfile)

## Convert Notebook to Script

The following cell will use `jupyter nbconvert` to build an `checklpp.py` which runs the script outlined in this notebook.

In [ ]:
%%javascript
var script_file = 'checklpp.py';

var notebook_name = window.document.getElementById('notebook_name').innerHTML;
var nbconvert_command = 'jupyter nbconvert --stdout --to script ' + notebook_name;

var grep_command = "grep -v '^#' | grep -v -F get_ipython | sed '/^$/N;/^\\n$/D'";
var command = '!' + nbconvert_command + ' | ' + grep_command + ' > ' + script_file;

if (Jupyter.notebook.kernel) {
    Jupyter.notebook.kernel.execute(command);
}